# Function for Phase and detuning correction

In [1]:
import numpy as np


def compute_phase_and_detuning_180Hz(pulse_train, fractions, pi_t = [19.470, 35.554, 41.166, 30.108, 39.326]):

    def line_signal(t, 
                    A1=0.000273203217587317, phi1=-0.7165710705760902,
                    A2=6.842623973133531e-05, phi2=-7.7358413871655065,
                    offset=0.0002890819550014299):
        return (
            2*np.pi*A1 * np.sin(2 * np.pi * 60 * t + phi1) +
            2*np.pi*A2 * np.sin(2 * np.pi * 180 * t + phi2) +
            offset
        ) - (
            2*np.pi*A1 * np.sin(2 * np.pi * 60 * 0 + phi1) +
            2*np.pi*A2 * np.sin(2 * np.pi * 180 * 0 + phi2) +
            offset
        )
    
    
    def analytical_integral(T, 
                    A1=0.000273203217587317, phi1=-0.7165710705760902,
                    A2=6.842623973133531e-05, phi2=-7.7358413871655065,
                    offset=0.0002890819550014299):
    
        int_60 = (A1 / 60) * (np.cos(phi1) - np.cos(2 * np.pi * 60 * T + phi1))
    
        int_180 = (A2 / 180) * (np.cos(phi2) - np.cos(2 * np.pi * 180 * T + phi2))
    
        t0 = (2 * np.pi * A1 * np.sin(2 * np.pi * 60 * 0 + phi1) +
              2 * np.pi * A2 * np.sin(2 * np.pi * 180 * 0 + phi2) +
              offset)
    
        int_offset = (offset - t0) * T
        return int_60 + int_180 + int_offset

    def compute_pi_times(pi_t):
        transition_strengths = np.loadtxt(
            'Z:\\Lab Data\\Phase_and_freq_correction_180Hz\\Transition_strengths_4p216.txt', delimiter=','
        )
        transition_strengths[transition_strengths == 0] = np.nan
    
        # pi_t = np.array([19.470, 35.554, 41.166, 30.108, 39.326])
        strengths = np.array([
            transition_strengths[23, 0], transition_strengths[14, 0],
            transition_strengths[17, 4], transition_strengths[16, 4], transition_strengths[15, 4]
        ])
    
        factors = np.array(pi_t) * strengths
        Fs = [1, 2, 3, 4]
        row_labels = [[i, i - j] for i in Fs for j in range(2 * i + 1)]
        col_labels = [-2, -1, 0, 1, 2]
    
        pi_times = np.zeros((24, 5))
        for i in range(np.shape(transition_strengths)[0]):
            for j in range(np.shape(transition_strengths)[1]):
                if not np.isnan(transition_strengths[i, j]):
                    delta_m = (row_labels[i][1] - col_labels[j]) + 2
                    pi_times[i, j] = factors[delta_m] / transition_strengths[i, j]
        
        return pi_times

    def get_pi_times(transitions,matrix):
        pi_times_list = []
        for transition in transitions:
            row_label = [transition[1],transition[2]]
            Fs = [1,2,3,4]
            states = []
            for i in Fs:
                for j in range(2*i+1):
                    mF = i-j
                    states.append([i,mF])
        
            row_labels = states
        
            col_label = transition[0]
        
            # Find the index of the row label
            row_index = next((i for i, label in enumerate(row_labels) if label == row_label), None)
            # Find the index of the column label
            col_index = col_labels.index(col_label)
            
            if row_index is not None and col_index in range(len(col_labels)):
                pi_times_list.append(matrix[row_index, col_index])
            else:
                pi_times_list.append(np.nan)
    
        return pi_times_list
    
    def get_pulse_schedule(rabi_freqs, fractions):
        if len(rabi_freqs) != len(fractions):
            raise ValueError(f"rabi_freqs {len(rabi_freqs)} and fractions {len(fractions)} must have the same length.")
        times = []
        t_current = 0.0  # start time in microseconds
        for Omega, frac in zip(rabi_freqs, fractions):
            if not 0 <= frac <= 1:
                raise ValueError(f"Fraction must be between 0 and 1, got {frac}.")
            # Calculate the rotation angle and pulse duration (in microseconds)
            theta = 2.0 * np.arcsin(np.sqrt(frac))
            t_pulse = theta / Omega if Omega > 0 else 0.0
            t_start = t_current
            t_end = t_current + t_pulse
            times.append((t_start, t_end))
            t_current = t_end
        return times

    pi_times_train = get_pi_times(pulse_train,compute_pi_times(pi_t))
    rabi_frequencis_list = np.pi/np.array(pi_times_train)

    sens_matrix = np.loadtxt('Z:\Lab Data\Phase_and_freq_correction_180Hz\sensitivities_4p216.txt',delimiter=',')
    sens_list = get_pi_times(pulse_train, sens_matrix)
    
    schedule = get_pulse_schedule(rabi_frequencis_list, fractions)
    pulses_sec = [(start * 1e-6, end * 1e-6) for (start, end) in schedule]
    # print(schedule)
    integrated_values = []
    detuning_values = []

    # For each pulse, compute the analytical integration and instantaneous detuning at the pulse start.
    for (start_s, _) in pulses_sec:
        integ = analytical_integral(start_s)
        integrated_values.append(integ)
        det = line_signal(start_s)
        detuning_values.append(det)
    
    # Convert to numpy arrays for vectorized operations.
    # print(integrated_values)
    integrated_values = np.array(integrated_values)
    detuning_values = np.array(detuning_values)
    sens_array = np.array(sens_list)
    
    # Compute phase (scaled integrated detuning) and instantaneous detuning.
    phase_180Hz = integrated_values * 1e6 * sens_array
    detuning_180Hz = detuning_values * sens_array
    
    return phase_180Hz, detuning_180Hz


## Visualization 

### Helper Functions

In [2]:
import numpy as np 
from scipy.io import loadmat

transition_strengths = np.loadtxt('Z:\Lab Data\Phase_and_freq_correction_180Hz\Transition_strengths_4p216.txt',delimiter=',')  # Assumes space-delimited file
transition_strengths[transition_strengths == 0] = np.nan
# print(transition_strengths)


pi_t = [18.622, 31.746, 37.865, 27.596, 36.429]
strengths = np.array([transition_strengths[23,0],transition_strengths[14,0],transition_strengths[17,4],transition_strengths[16,4],transition_strengths[15,4]])
pitime_n2 = pi_t[0] # [-2, 4, -4]
pitime_n1 = pi_t[1] # [-2, 3, -3]
pitime_0 = pi_t[2] # [2, 4, 2]
pitime_p1 = pi_t[3] # [2, 4, 3]
pitime_p2 = pi_t[4] # [2, 4, 4]

pi_t = np.array([pitime_n2,pitime_n1,pitime_0,pitime_p1,pitime_p2])
factors = pi_t * strengths
Fs = [1,2,3,4]
row_labels = []
for i in Fs:
    for j in range(2*i+1):
        mF = i-j
        row_labels.append([i,mF])
col_labels = [-2, -1, 0, 1, 2]

pi_times = np.zeros((24,5))
for i in range(np.shape(transition_strengths)[0]):
    for j in range(np.shape(transition_strengths)[1]):
        # print(i,j)
        if not np.isnan(transition_strengths[i,j]):

            delta_m = (row_labels[i][1]-col_labels[j])+2

            pi_times[i,j] = factors[delta_m]/transition_strengths[i,j]
print(pi_times)
def get_pi_times(transitions,matrix = pi_times):
    pi_times_list = []
    for transition in transitions:
        row_label = [transition[1],transition[2]]
        Fs = [1,2,3,4]
        states = []
        for i in Fs:
            for j in range(2*i+1):
                mF = i-j
                states.append([i,mF])
    
        row_labels = states
    
        col_label = transition[0]
    
        # Find the index of the row label
        row_index = next((i for i, label in enumerate(row_labels) if label == row_label), None)
        # Find the index of the column label
        col_index = col_labels.index(col_label)
        
        if row_index is not None and col_index in range(len(col_labels)):
            pi_times_list.append(matrix[row_index, col_index])
        else:
            pi_times_list.append(np.nan)

    return pi_times_list

def get_pulse_schedule(rabi_freqs, fractions):
    if len(rabi_freqs) != len(fractions):
        raise ValueError("rabi_freqs and fractions must have the same length.")
    
    times = []
    t_current = 0.0  # microseconds (start at time=0)
    
    for Omega, frac in zip(rabi_freqs, fractions):
        if not 0 <= frac <= 1:
            raise ValueError(f"Fraction must be between 0 and 1, got {frac}.")
        
        # Calculate the rotation angle and the pulse duration
        theta = 2.0 * np.arcsin(np.sqrt(frac))   # in radians
        t_pulse = theta / Omega if Omega > 0 else 0.0  # microseconds (if Omega in MHz)
        
        # The pulse starts at t_current, ends at t_current + t_pulse
        t_start = t_current
        t_end = t_current + t_pulse
        
        times.append((t_start, t_end))
        t_current = t_end  # next pulse starts after this one finishes
    
    return times

mat_data = loadmat('sensitivity_matrix_4p216G.mat')

matrix_sen_24x5 = mat_data['S']
matrix_sen_24x5[np.isnan(transition_strengths)] = np.nan
np.savetxt('sensitivities_4p216.txt',matrix_sen_24x5, delimiter=',')
sens_matrix = np.loadtxt('Z:\Lab Data\Phase_and_freq_correction_180Hz\sensitivities_4p216.txt',delimiter=',')
print(sens_matrix)
# print(matrix_sen_24x5)

[[   0.          211.9322501   134.46824927  170.80710206  249.8197383 ]
 [ 158.67999176  301.16549727  921.45526918  253.17624047  108.98500926]
 [ 139.52333084  148.14607451  181.16403349  171.87311245    0.        ]
 [   0.            0.          108.23055675   69.68854243  108.16046645]
 [   0.           93.77573445  268.8374713   144.6413757    86.74807471]
 [ 122.70543703  116.4129164    96.40360429  329.79550561   65.47951644]
 [  63.61091827  314.75453041  147.92715683   57.05244149    0.        ]
 [  80.1717235    80.57107542   75.00681767    0.            0.        ]
 [   0.            0.            0.           42.44279842  816.89796011]
 [   0.            0.           52.18745061   47.73855147  213.54373172]
 [   0.           78.71729484   34.64748359  127.12049517  190.86212989]
 [ 156.52264106   36.83116196   50.89949487 2802.28634035  170.05153014]
 [  54.1229026    37.67897927  110.12364584  156.61516797    0.        ]
 [  39.54078846   48.26913817  577.36344243    0.  

FileNotFoundError: [Errno 2] No such file or directory: 'sensitivity_matrix_4p216G.mat'